In [1]:
from keras.applications import VGG19
from keras import models, layers, optimizers, regularizers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [ ]:
# !unzip data/hot-dog-not-hot-dog.zip -d data/ # sample code

## Importing the Data and Preprocessing 

In [2]:
data_te = ImageDataGenerator(rescale = 1./255).flow_from_directory('DATASET/TEST', 
                                                                   target_size = (224, 224), 
                                                                   batch_size = 2513,
                                                                   seed = 123)

data_tr = ImageDataGenerator(rescale = 1./255).flow_from_directory('DATASET/TRAIN', 
                                                                   target_size = (224, 224),  
                                                                   batch_size = 22564,
                                                                   seed = 123)

Found 2513 images belonging to 2 classes.
Found 22564 images belonging to 2 classes.


In [3]:
mod_te = ImageDataGenerator(rescale = 1./255, 
                            rotation_range = 360, 
                            width_shift_range = 0.3, 
                            height_shift_range = 0.3, 
                            brightness_range = [0.2, 1.0], 
                            horizontal_flip = True, 
                            vertical_flip = True, 
                            zoom_range = [0.5, 1.5]).flow_from_directory('DATASET/TEST', 
                                                                   target_size = (224, 224), 
                                                                   batch_size = 625,
                                                                   seed = 123)

mod_tr = ImageDataGenerator(rescale = 1./255, 
                            rotation_range = 360, 
                            width_shift_range = 0.3, 
                            height_shift_range = 0.3, 
                            brightness_range = [0.2, 1.0], 
                            horizontal_flip = True, 
                            vertical_flip = True, 
                            zoom_range = [0.5, 1.5]).flow_from_directory('DATASET/TRAIN', 
                                                                   target_size = (224, 224), 
                                                                   batch_size = 1000,
                                                                   seed = 123)

Found 2513 images belonging to 2 classes.
Found 22564 images belonging to 2 classes.


In [4]:
images_te, labels_te = next(data_te)
images_tr, labels_tr = next(data_tr)

In [5]:
images_mod_te, labels_mod_te = next(mod_te)
images_mod_tr, labels_mod_tr = next(mod_tr)

In [6]:
print(images_te.shape)
print(images_tr.shape)

(2513, 224, 224, 3)
(22564, 224, 224, 3)


In [7]:
print(images_mod_te.shape)
print(images_mod_tr.shape)

(625, 224, 224, 3)
(1000, 224, 224, 3)


In [8]:
print(labels_te.shape)
print(labels_tr.shape)

(2513, 2)
(22564, 2)


In [9]:
print(labels_mod_te.shape)
print(labels_mod_tr.shape)

(625, 2)
(1000, 2)


In [10]:
images = np.concatenate((images_tr, images_te, images_mod_tr, images_mod_te))

In [11]:
images.shape

(26702, 224, 224, 3)

In [12]:
labels = np.concatenate((labels_tr[:,0], labels_te[:,0], labels_mod_tr[:,0], labels_mod_te[:, 0]))

In [13]:
labels.shape

(26702,)

In [14]:
X_model, X_test, y_model, y_test = train_test_split(images, labels, test_size = 0.3, random_state = 123)

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X_model, y_model, test_size = 0.3, random_state = 123)

## Building CNN Models 

In [16]:
cnn_vgg_base = VGG19(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
model_1 = models.Sequential()
model_1.add(cnn_vgg_base)
model_1.add(layers.Flatten())
model_1.add(layers.Dense(1024, activation='relu', kernel_regularizer = regularizers.l2(0.01)))
model_1.add(layers.Dense(1024, activation='relu', kernel_regularizer = regularizers.l2(0.01)))  
model_1.add(layers.Dense(512, activation='relu', kernel_regularizer = regularizers.l2(0.01)))  
model_1.add(layers.Dense(1, activation='sigmoid'))

In [16]:
for layer in model_1.layers:
    print(layer.name, layer.trainable)

vgg19 True
flatten_1 True
dense_1 True
dense_2 True
dense_3 True
dense_4 True


In [17]:
cnn_vgg_base.trainable = False

In [18]:
for layer in model_1.layers:
    print(layer.name, layer.trainable)

vgg19 False
flatten_1 True
dense_1 True
dense_2 True
dense_3 True
dense_4 True


In [18]:
model_1.compile(loss = 'binary_crossentropy', optimizer = optimizers.Adam(), metrics = ['acc'])

In [ ]:
history_1 = model_1.fit(X_train, 
                                y_train, 
                                batch_size = 200, 
                                epochs = 10, 
                                validation_data = (X_val, y_val), )

Train on 13083 samples, validate on 5608 samples
Epoch 1/10
